In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [ ]:
# Import raw data
calander = pd.read_csv('data/calendar.csv')
df=pd.read_csv('data/sales_train_validation.csv')
sell_prices = pd.read_csv('data/sell_prices.csv')

In [ ]:
calander.head()

In [ ]:
sell_prices.head()

In [ ]:
df.head()

In [ ]:
train_df = pd.read_csv('train_val_df_2011.csv')

In [2]:
def create_train_data(ds,de):
    # Import raw data
    calander = pd.read_csv('data/calendar.csv')
    df=pd.read_csv('data/sales_train_validation.csv')
    sell_prices = pd.read_csv('data/sell_prices.csv')
    calander = calander[(calander['year'] >= ds) & (calander['year'] <de)]
    # Before one row represents all sales for one item (30490 rows)
    # After one row represents one day's sales for one item (58327370 rows = 30490 * 1913)
    cat_cols = ['id','item_id','dept_id','cat_id','store_id','state_id']
    numcols = [f"d_{day}" for day in range(1,1914)]
    df = pd.melt(df, id_vars=cat_cols, value_vars = numcols, var_name='d',value_name='sales')
    
    # Merge with calander
    df_train = df.merge(calander,on='d')
    # removing dept_id and store from features, not enough training data atm for these
    features = ['item_id','cat_id','state_id','wday','month','year']
    cat = ['item_id','cat_id','state_id']
    target = 'sales'
    for c in cat:
        df_train[c] = df_train[c].astype('category').cat.codes
    
    df_train['d'] =  pd.to_numeric(df_train['d'].str.split('_',expand=True)[1])
    train_df = df_train[df_train.d < df_train['d'].unique()[-28]]
    val_df= df_train[df_train.d >= df_train['d'].unique()[-28]]
    X = train_df[features].values
    X_val = val_df[features].values
    y=train_df[target].values
    y_val=val_df[target].values
    train_data = lgb.Dataset(X,label=y,feature_name = features, categorical_feature = cat,free_raw_data=False)
    val_data = lgb.Dataset(X_val,label=y_val)
    
    return train_data, val_data

In [3]:
train_data,val_data = create_train_data(ds=2011,de=2016)

MemoryError: 

In [ ]:
paramaters = {'objective':'rmse','metric':'rmse','num_leaves':31}

In [ ]:
model = lgb.train(paramaters,train_data,valid_sets = val_data,num_boost_round = 1000, early_stopping_rounds=100)

Submission File
Each row contains an id that is a concatenation of an item_id and a store_id, which is either validation (corresponding to the Public leaderboard), or evaluation (corresponding to the Private leaderboard). You are predicting 28 forecast days (F1-F28) of items sold for each row. For the validation rows, this corresponds to d_1914 - d_1941, and for the evaluation rows, this corresponds to d_1942 - d_1969. (Note: a month before the competition close, the ground truth for the validation rows will be provided.)

The files must have a header and should look like the following:

id,F1,...F28

HOBBIES_1_001_CA_1_validation,0,...,2 

HOBBIES_1_002_CA_1_validation,2,...,11

...
HOBBIES_1_001_CA_1_evaluation,3,...,7

HOBBIES_1_002_CA_1_evaluation,1,...,4

In [ ]:
# Creates test data for model.
# ds {int} Date start, inclusive. e.g 1914 (for this comp)
# de {int} Date end, inclusive. e.g 1941 (for this comp)
#
def create_test_data(ds=1914,de=1941):
    calander = pd.read_csv('data/calendar.csv')
    df=pd.read_csv('data/sales_train_validation.csv')
    sell_prices = pd.read_csv('data/sell_prices.csv')
    features = ['item_id','cat_id','state_id','wday','month','year']
    df_features = ['item_id','cat_id','state_id']
    cal_features = ['wday','month','year']
    test_ids = df['id'].unique()
    df_test = df[df_features]
    
    cat = ['item_id','cat_id','state_id']
    for c in cat:
        df_test[c] = df_test[c].astype('category').cat.codes
    
    #test_day_columns=['F'+str(i) for i in range(1,28)]
    calander['d'] =  pd.to_numeric(calander['d'].str.split('_',expand=True)[1])
    calander = calander[(calander.d >= ds) & (calander.d <=de)]
    calander['key'] = 0
    df_test['key'] = 0
    merged = df_test.merge(calander, on='key')
    df_test = merged[features]
    return df_test, df['id']
    

In [ ]:
def create_28_day_test_submission(test_df,ids,model):
    y_pred = model.predict(test_df.values)
    y_grp = [y_pred[i:i+28] for i  in range(0, len(y_pred), 28)]
    test_day_columns=['F'+str(i) for i in range(1,29)]
    df_sub = pd.DataFrame(columns=test_day_columns,data=y_grp)
    df_sub['id'] = ids
    return df_sub[['id',*test_day_columns]]
    

In [ ]:
val_test_data,val_ids = create_test_data(ds=1914,de=1941)

In [ ]:
# Need to change ids from validation to evaluation
eval_test_data, eval_ids = create_test_data(ds=1942,de=1969)

In [ ]:
eval_ids = eval_ids.str.replace('validation','evaluation')

In [ ]:
df_val_sub = create_28_day_test_submission(val_test_data,val_ids,model)
df_eval_sub = create_28_day_test_submission(eval_test_data,eval_ids,model)
df_val_sub.head()

In [ ]:
final_submission = pd.concat([df_val_sub,df_eval_sub])

In [ ]:
final_submission.to_csv('sub_1.csv',index=False)